# 3.1 Path of steepest ascent



In [ ]:
first.order <- function(x1, x2) {
  20 + 3*x1 - 1.5*x2
}


In [ ]:
x1 <- x2 <- seq(-2, 3.5, length=1000)
g <- expand.grid(x1, x2)
yhat <- matrix(first.order(g[,1], g[,2]), ncol=length(x2))
D <- rbind(c(-1,-1), c(-1,1), c(1,-1), c(1,1))
D


In [ ]:
cols <- heat.colors(128)
par(pty="s")
image(x1, x2, yhat, col=cols)
contour(x1, x2, yhat, add=TRUE)
points(D, pch=19)
points(0, 0)
arrows(0, 0, 3, -1.5)


In [ ]:
par(pty="s")
image(x1, x2, yhat, col=cols)
contour(x1, x2, yhat, add=TRUE)
points(D, pch=19)
points(0, 0)
arrows(0, 0, 3, -1.5)
library(plotrix)
draw.circle(0, 0, 1)
text(1, 0, "r1")
draw.circle(0, 0, 1.75)
text(1.75, 0, "r2 > r1")


In [ ]:
plasma <- read.table("dataset/plasma.txt", header=TRUE)
plasma


In [ ]:
library(magrittr)
fit.int <- lm(etch ~ x1*x2, data=plasma)
summary(fit.int)
#summary(fit.int)$coefficients %>% signif(digits=4)


In [ ]:
fit <- lm(etch ~ x1 + x2, data=plasma)
summary(fit)
coef(fit)


In [ ]:
AIC(fit, fit.int)
BIC(fit, fit.int)


In [ ]:
b1 <- coef(fit)[2]
b2 <- coef(fit)[3]
delta2 <- abs(b2/b1)
delta2


In [ ]:
Dnew <- data.frame(x1=-1:-3, x2=(1:3)*delta2)
Dnew


In [ ]:
x1 <- seq(-4, 1.5, length=100)
# x2 <- seq(-1.5, 4*delta2, length=100) 
x2 <- seq(-1.5, 4, length=100)  # aspect ratio = 1 유지
g <- expand.grid(x1=x1, x2=x2)
yhat <- matrix(predict(fit, newdata=g), ncol=length(x2))


In [ ]:
par(pty="s")
image(x1, x2, yhat, col=cols)
contour(x1, x2, yhat, add=TRUE)
points(plasma$x1, plasma$x2, pch=19)
points(0, 0)
arrows(0, 0, -3.5, 3.5*delta2)
points(Dnew)


In [ ]:
plasma.delta <- read.table("dataset/plasma_delta.txt", header=TRUE)
plasma.delta$p.etch <- predict(fit, newdata=plasma.delta[,4:5])
plasma.delta


In [ ]:
max(plasma$etch)



In [ ]:
r <- cbind(c(1, 2), c(100, 150), c(500, 1000), c(75, 120))
colnames(r) <- c("vel", "temp", "mpress", "bpress")
r


In [ ]:
base <- r[1,] + (r[2,] - r[1,])/2
base


In [ ]:
b <- c(-5.24, -6.22, -1.21, -1.07)
delta <- b/b[1]
delta


In [ ]:
path <- rbind(0, apply(matrix(rep(delta, 4), ncol=4, byrow=TRUE), 2, cumsum))
colnames(path) <- paste0("x", 1:4)
rownames(path) <- paste0("Base +", 0:4, "Δ")
path


In [ ]:
dnat <- delta*(r[2,] - r[1,])/2
dnat


In [ ]:
matrix(rep(base, 4), ncol=4, byrow=TRUE)



In [ ]:
pnat <- rbind(base, matrix(rep(base, 4), ncol=4, byrow=TRUE) + 
                apply(matrix(rep(dnat, 4), ncol=4, byrow=TRUE), 2, cumsum))
colnames(pnat) <- c("ft/sec", "°C", "psi", "psi")
rownames(pnat) <- rownames(path)
signif(cbind(path, pnat), digits=5)


In [ ]:
x1 <- seq(0, 1, length=1000)
x2 <- sqrt(1 - x1^2)
x1 <- c(x1, x1)
x2 <- c(x2, -x2)
ci95 <- (3*x1 - 1.5*x2)^2 >= 9.3225

par(pty="s")
plot(0, type="n", xlim=c(-2, 3.5), ylim=c(-2, 3.5), xlab="x1", ylab="x2")
points(D, pch=19)
points(0, 0)
arrows(0, 0, 3, -1.5)
points(x1, x2, col=2 + ci95, pch=19, cex=0.5)
#plot(x1, x2)


In [ ]:
theta <- asin(sqrt(0.25*7.71*(1/11.25)))
theta


In [ ]:
O <- c(4/3, 8/3)



In [ ]:
par(pty="s")
plot(0, 0, type="n", xlab="x1", ylab="x2", xlim=c(0, 4), ylim=c(0, 5))
arrows(0, 0, O[1], O[2], col=2, lwd=3)
arrows(O[1], O[2], 3, 2 + 3/2, col=3, lwd=3)
text(O[1] - 0.1, O[2] + 0.2, "O")
abline(0, 2, lty=2)
abline(2, 0.5, lty=3)
text(1.5, 1, "steepest ascent", col=2)
text(2.5, 2.7, "constrained", col=3)


In [ ]:
upper <- c(150, 100, 40)
lower <- c(100, 50, 20)
scale <- (upper - lower) / 2
shift <- scale + lower
toxi <- data.frame(scale=scale, shift=shift)
toxi


In [ ]:
c <- c(-300, 25, 25, 0)



In [ ]:
b <- c(150, 1.7, 0.8, 0.5)



In [ ]:
rhoo <- -c[1]/sum(c[-1]*b[-1])
rhoo


In [ ]:
xo <- rhoo*b[-1]
xo


In [ ]:
d <- as.numeric(coef(lm(b[-1] ~ c[-1] - 1)))
d


In [ ]:
hpath <- function(lambda, b, c, rhoo, d)
 {
  ## steepest ascent up to one step past the constraint boundary
  delta <- b/b[1]
  path <- matrix(0, ncol=length(b), nrow=1)
  while(1) {
    lpath <- nrow(path)
    path <- rbind(path, path[lpath,] + delta)
    if(c[1] + sum(path[lpath + 1,]*c[-1]) > 0) break
  }
    
  ## intersection point plus steps along the modified portion
  cpath <- rhoo*b
  for(i in 1:length(lambda)) {
    cpath <- rbind(cpath, rhoo*b + lambda[i]*(b - d*c[-1]))
  }
    
  ## pasting the hybrid path together and naming the rows and columns
  path <- rbind(path[1:lpath,], cpath)
  colnames(path) <- paste("x", 1:length(b), sep="")
  rownames(path) <- c(rep("u", lpath), "o", rep("c", length(lambda)))
  return(path)
 }


In [ ]:
lambda <- c(1,2,3,4)
path <- hpath(lambda, b[-1], c, rhoo, d)


In [ ]:
A <- matrix(rep(toxi[,1], nrow(path)), ncol=ncol(path), byrow=TRUE)
B <- matrix(rep(toxi[,2], nrow(path)), ncol=ncol(path), byrow=TRUE)
pathxi <- A * path + B
colnames(pathxi) <- paste("xi", 1:3, sep="")
pathxi


In [ ]:
pathxi[4:5,]



## 3.2 Second-order response surfaces



In [ ]:
second.order <- function(x1, x2) {
  100 + 5*x1 + 10*x2 - 8*x1^2 - 12*x2^2 - 12*x1*x2
}


In [ ]:
x1 <- x2 <- seq(-4, 5, length=100)
g <- expand.grid(x1, x2)
y <- matrix(second.order(g[,1], g[,2]), ncol=length(x2))


In [ ]:
persp(x1, x2, y, theta=30, phi=30, zlab="eta", expand=0.75, lwd=0.25)



In [ ]:
par(pty="s")
image(x1, x2, y, col=heat.colors(128))
contour(x1, x2, y, add=TRUE)


In [ ]:
dy <- rbind(c(16, 12), c(12, 24))
xh <- solve(dy, c(5, 10))
#yh <- 100 + 5*xh[1] + 10*xh[2] - 8*xh[1]^2 - 12*xh[2]^2 - 12*xh[1]*xh[2]
yh <- second.order(xh[1], xh[2])
c(x1=xh[1], x2=xh[2], y=yh)


In [ ]:
b0 <- 100
b <- c(5, 10)
B <- matrix(c(-8, -12/2, -12/2, -12), ncol=2, byrow=TRUE)
B


In [ ]:
xs <- -0.5*solve(B) %*% b
ys <- b0 + 0.5*t(xs) %*% b
sols <- rbind(h = c(xh, yh), s=c(xs, ys))
colnames(sols) <- c("x1", "x2", "y")
sols


In [ ]:
E <- eigen(B)
lambda <- E$values
o <- order(abs(lambda), decreasing=TRUE)
lambda <- lambda[o]
lambda


In [ ]:
V <- E$vectors[,o]
V


In [ ]:
par(pty="s")
image(x1, x2, y, col=cols)
contour(x1, x2, y, add=TRUE)
lines(c(-V[1,1], V[1,1])*10 + xs[1], c(-V[2,1], V[2,1])*10 + xs[2], lty=2)
lines(c(-V[1,2], V[1,2])*10 + xs[1], c(-V[2,2], V[2,2])*10 + xs[2], lty=2)


In [ ]:
chem <- read.table("dataset/chemical.txt", header=TRUE)
chem
uchem <- unique(chem[,1:2])
uchem


In [ ]:
library(dplyr)
chem %>% 
  group_by(x1, x2) %>% 
  summarize(n = n(), .groups="drop") %>% 
  ungroup() #%>% 
  #pull(n) -> reps
#reps


In [ ]:
reps <- apply(uchem, 1, function(x) {
  sum(apply(chem[,1:2], 1, function(y) { all(y == x) }))
})
reps


In [ ]:
par(pty="s")
plot(uchem, type="n")
text(uchem, labels=reps)


In [ ]:
X <- with(chem, data.frame(x1=x1, x2=x2, x11=x1^2, x22=x2^2, x12=x1*x2))
y <- chem$y
fit <- lm(y ~ ., data=X)
summary(fit)


In [ ]:
r <- cbind(c(200, 250), c(15, 25))
d <- (r[2,] - r[1,])/2
xi1 <- seq(min(chem$temp), max(chem$temp), length=100)
xi2 <- seq(min(chem$conc), max(chem$conc), length=100)
xi <- expand.grid(xi1, xi2)
x <- cbind((xi[,1] - r[2,1] + d[1])/d[1], (xi[,2] - r[2,2] + d[2])/d[2])


In [ ]:
XX <- data.frame(x1=x[,1], x2=x[,2], x11=x[,1]^2, x22=x[,2]^2, x12=x[,1]*x[,2])
p <- predict(fit, newdata=XX)
head(p)


In [ ]:
par(pty="s")
xlab <- "Temperature (°C)"
ylab <- "Concentration (%)"
image(xi1, xi2, matrix(p, nrow=length(xi1)), col=cols, xlab=xlab, ylab=ylab)
contour(xi1, xi2, matrix(p, nrow=length(xi1)), add=TRUE)


In [ ]:
b <- coef(fit)[2:3]
B <- matrix(NA, nrow=2, ncol=2)
diag(B) <- coef(fit)[4:5]
B[1, 2] <- B[2, 1] <- coef(fit)[6]/2
xs <- -1/2*solve(B, b)
xs


In [ ]:
xis <- xs*d + (r[2,] - d)
xis


In [ ]:
E <- eigen(B)
E


In [ ]:
lambda <- E$values
o <- order(abs(lambda), decreasing=TRUE)
lambda <- lambda[o]
V <- E$vectors[,o]
Vxi <- V
for(j in 1:ncol(Vxi)) Vxi[,j] <- Vxi[,j]*d*10


In [ ]:
par(pty="s")
image(xi1, xi2, matrix(p, nrow=length(xi1)), col=cols, xlab=xlab, ylab=ylab)
contour(xi1, xi2, matrix(p, nrow=length(xi1)), add=TRUE)
lines(c(-Vxi[1,1], Vxi[1,1])+xis[1], c(-Vxi[2,1], Vxi[2,1])+xis[2], lty=2)
lines(c(-Vxi[1,2], Vxi[1,2])+xis[1], c(-Vxi[2,2], Vxi[2,2])+xis[2], lty=2)
points(xis[1], xis[2])
text(xis[1], xis[2], "xs", pos=4)


In [ ]:
ys <- as.numeric(coef(fit)[1] + 0.5*t(xs) %*% b)
ys


In [ ]:
rr <- read.table("dataset/risingridge.txt", header=TRUE)
rr


In [ ]:
rr$A2 <- rr$A^2
rr$B2 <- rr$B^2
rr$AB <- rr$A*rr$B
rr


In [ ]:
fit <- lm(y ~ ., data=rr)
summary(fit)


In [ ]:
b <- coef(fit)[2:3]
B <- matrix(NA, nrow=2, ncol=2)
diag(B) <- coef(fit)[4:5]
B[1,2] <- B[2,1] <- coef(fit)[6]/2
xs <- -1/2*solve(B, b)
xs


In [ ]:
apply(rr[,1:2], 2, range)



In [ ]:
E <- eigen(B)
lambda <- E$values
o <- order(abs(lambda), decreasing=TRUE)
V <- E$vectors[,o]*20
lambda <- lambda[o]
lambda


In [ ]:
ys <- coef(fit)[1] + 0.5*t(xs) %*% b
ys


In [ ]:
x <- seq(-6, 6, length=100)
xx <- expand.grid(x, x)
XX <- data.frame(A=xx[,1], B=xx[,2], A2=xx[,1]^2, B2=xx[,2]^2, AB=xx[,1]*xx[,2])
p <- predict(fit, newdata=XX)
par(pty="s")
image(x, x, matrix(p, nrow=length(x)), col=cols, xlab="A", ylab="B")
contour(x, x, matrix(p, nrow=length(x)), add=TRUE)
lines(c(-V[1,1], V[1,1]) + xs[1], c(-V[2,1], V[2,1]) + xs[2], lty=2)
lines(c(-V[1,2], V[1,2]) + xs[1], c(-V[2,2], V[2,2]) + xs[2], lty=2)
polygon(c(1,1,-1,-1), c(1,-1,-1,1), lty=3)
text(0, -0.5, "design region", cex=0.5)
points(xs[1], xs[2])
text(xs[1], xs[2], "xs", pos=4)


In [ ]:
saddle <- read.table("dataset/saddle.txt", header=TRUE)
saddle <- cbind(saddle[,-5]^2, model.matrix(~ .^2 - 1, saddle[,-5]), y=saddle[,5])
names(saddle)[1:4] <- paste("x", 1:4, 1:4, sep="")
names(saddle)[9:14] <- sub(":x", "", names(saddle)[9:14])
saddle <- saddle[c(5:8, 1:4, 9:15)]

saddle


In [ ]:
fit <- lm(y ~ ., data=saddle)
summary(fit)


In [ ]:
b <- coef(fit)[2:5]
B <- matrix(NA, nrow=4, ncol=4)
diag(B) <- coef(fit)[6:9]
i <- 10
for (j in 1:3) {
  for (k in (j+1):4) {
    B[j,k] <- B[k,j] <- coef(fit)[i]/2
    i <- i + 1
  }
}


In [ ]:
xs <- -1/2*solve(B, b)
xs


In [ ]:
apply(saddle[,1:4], 2, range)



In [ ]:
E <- eigen(B)
lambda <- E$values
o <- order(abs(lambda), decreasing=TRUE)
lambda <- lambda[o]
lambda


In [ ]:
mul <- max(lambda)
mul
x <- solve(B - mul*diag(4), -b/2)
x


In [ ]:
f <- function(mu, R2=1) {
  x <- solve(B - mu*diag(4), -b/2)
  R2 - t(x) %*% x
}


In [ ]:
mu <- uniroot(f, c(mul, 10*mul), R2=1.4^2)$root
mu


In [ ]:
x <- solve(B - mu*diag(4), -b/2)
x


In [ ]:
drop(sqrt(t(x) %*% x))



In [ ]:
mus <- rs <- seq(0.1, 2, length=20)
xp <- matrix(NA, nrow=length(rs), ncol=4)
colnames(xp) <- c("x1", "x2", "x3", "x4")
for(i in 1:length(rs)) {
  mus[i] <- uniroot(f, c(mul, 100*mul), R2=rs[i]^2)$root
  xp[i,] <- solve(B - mus[i]*diag(4), -b/2)
}
xp <- rbind(rep(0, 4), xp)
rs <- c(0, rs)
mus <- c(Inf, mus)
mus


In [ ]:
Xp <- data.frame(xp)
Xp <- cbind(Xp^2, model.matrix(~ .^2 - 1, Xp))
names(Xp)[1:4] <- paste("x", 1:4, 1:4, sep="")
names(Xp)[9:14] <- sub(":x", "", names(Xp)[9:14])
Xp <- Xp[c(5:8, 1:4, 9:14)]
Xp


In [ ]:
p <- predict(fit, newdata=Xp, se.fit=TRUE)



In [ ]:
cbind(R=rs, mu=mus, data.frame(pred=p$fit, se=p$se.fit), round(xp, 6))



In [ ]:
par(pty="s")
plot(rs, p$fit, type="b", ylim=c(20, 100), xlab="radius (R)", ylab="y.hat(x) & CIs")
lines(rs, p$fit + 2*p$se, col=2, lty=2)
lines(rs, p$fit - 2*p$se, col=2, lty=2)


In [ ]:
x12 <- seq(-2, 2, length=100)
g <- expand.grid(x12, x12)
Xp <- data.frame(x1=g[,1], x2=g[,2], x11=g[,1]^2, x22=g[,2]^2, x12=g[,1]*g[,2])
Xp


In [ ]:
## x3 = x4 = 0
Xp$x3 <- Xp$x33 <- Xp$x4 <- Xp$x44 <-Xp$x34 <- 0
Xp$x13 <- Xp$x14 <- Xp$x23 <- Xp$x24 <- Xp$x34 <- 0
p0 <- predict(fit, newdata=Xp, se.fit=TRUE)

## x3 = x4 = 1
Xp$x3 <- Xp$x33 <- Xp$x4 <- Xp$x44 <-Xp$x34 <- 1
Xp$x13 <- Xp$x14 <- Xp$X23 <- Xp$X24 <- Xp$X34 <- 1
p1 <- predict(fit, newdata=Xp, se.fit=TRUE)


In [ ]:
par(mfrow=c(1, 2), pty="s")
bs <- seq(min(p0$se.fit), max(p0$se.fit), length=129)
image(x12, x12, matrix(p0$se.fit, nrow=length(x12)), col=cols,
      breaks=bs, xlab="x1", ylab="x2", main="se.fit, x3 = x4 = 0")
contour(x12, x12, matrix(p0$se.fit, nrow=length(x12)), add=TRUE)
points(saddle[,1:2])
points(saddle[apply(saddle[,3:4] == c(0,0), 1, all), 1:2], pch=19)

image(x12, x12, matrix(p1$se.fit, nrow=length(x12)), col=cols,
      breaks=bs, xlab="x1", ylab="x2", main="se.fit, x3 = x4 = 1")
contour(x12, x12, matrix(p1$se.fit, nrow=length(x12)), add=TRUE)
points(saddle[,1:2])
points(saddle[apply(saddle[,3:4] == c(1, 1), 1, all), 1:2], pch=19)


In [ ]:
crdat <- read.table("dataset/confreg.txt", header=TRUE)
crdat


In [ ]:
crdat$x11 <- crdat$x1^2
crdat$x22 <- crdat$x2^2
crdat$x12 <- crdat$x1 * crdat$x2
crdat


In [ ]:
fit <- lm(y ~ ., data=crdat)
summary(fit)


In [ ]:
b <- coef(fit)[2:3]
B <- matrix(NA, nrow=2, ncol=2)
diag(B) <- coef(fit)[4:5]
B[1,2] <- B[2,1] <- coef(fit)[6]/2
xs <- -(1/2)*solve(B, b)
xs


In [ ]:
eigen(B)$values



In [ ]:
xx <- xx <- seq(-1.6, 1.6, length=200)
g <- expand.grid(xx, xx)
XX <- data.frame(x1=g[,1], x2=g[,2], x11=g[,1]^2, x22=g[,2]^2, x12=g[,1]*g[,2])
p <- as.numeric(predict(fit, newdata=XX))
head(p)


In [ ]:
par(pty="s")
image(xx, xx, matrix(p, nrow=length(xx)), col=cols, xlab="x1", ylab="x2")
contour(xx, xx, matrix(p, nrow=length(xx)), add=TRUE)
points(crdat$x1, crdat$x2, pch=20)
points(xs[1], xs[2])
text(xs[1], xs[2], "xs", pos=4)


In [ ]:
d <- rbind(c(b[1], 2*B[,1]), c(b[2], 2*B[,2]))
colnames(d) <- c("(Intercept)", "t1", "t2")
d


In [ ]:
d <- cbind(b, 2*B)
colnames(d) <- c("(Intercept)", "t1", "t2")
d


In [ ]:
s2 <- summary(fit)$sigma^2
s2


In [ ]:
X <- cbind(1, as.matrix(crdat[,-3]))
XtXi <- solve(t(X) %*% X)
XtXi


In [ ]:
Vard <- function(t1, t2, s2, XtXi) {
  v11 <- XtXi[2,2] + 4*t1^2*XtXi[4,4] + t2^2*XtXi[6,6]
  v22 <- XtXi[3,3] + t1^2*XtXi[6,6] + 4*t2^2*XtXi[5,5]
  v12 <- v21 <- 4*t1*t2*XtXi[4,5] + t1*t2*XtXi[6,6]
  v <- s2* matrix(c(v11,v12,v21,v22), ncol=2, byrow=TRUE)
  v
}


In [ ]:
CIqf <- function(t1, t2, s2, XtXi, b, B) {
  dt <- b + 2*B %*% c(t1, t2)
  V <- Vard(t1, t2, s2, XtXi)
  Vi <- solve(V)
  t(dt) %*% Vi %*% dt
}


In [ ]:
quadform <- rep(NA, nrow(g))
for(i in 1:nrow(g)) quadform[i] <- CIqf(g[i,1], g[i,2], s2, XtXi, b, B)


In [ ]:
ci90 <- quadform <= 2*qf(0.9, 2, nrow(X) - ncol(X))
ci95 <- quadform <= 2*qf(0.95, 2, nrow(X) - ncol(X))
par(pty="s")
image(xx, xx, matrix(ci90 + ci95, ncol=length(xx)), xlab="x1", ylab="x2", col=c("white", "lightgray", "darkgray"))
text(c(-0.2,-1), c(-0.5,1), c("90%","95%"))


In [ ]:
summary(fit)$r.squared



In [ ]:
crdat$y <- c(87.6, 86.5, 85.7, 86.9, 86.7, 86.8, 87.4, 86.7, 90.3, 91.0, 90.8)
fit2 <- lm(y ~ ., data=crdat)
summary(fit2)$r.squared


In [ ]:
b <- coef(fit)[2:3]
B <- matrix(NA, nrow=2, ncol=2)
diag(B) <- coef(fit)[4:5]
B[1,2] <- B[2,1] <- coef(fit)[6]/2
xs <- -(1/2)*solve(B, b)
s2 <- summary(fit2)$sigma^2
for(i in 1:nrow(g)) quadform[i] <- CIqf(g[i,1], g[i,2], s2, XtXi, b, B)


In [ ]:
ci95 <- quadform <= 2*qf(0.95, 2, nrow(X)-ncol(X))
par(pty="s")
image(xx, xx, matrix(ci95, ncol=length(xx)), xlab="x1", ylab="x2",
    col=c("white", "lightgray"))


In [ ]:
chem <- read.table("dataset/chemical.txt", header=TRUE)
X <- data.frame(x1=chem$x1, x2=chem$x2, x11=chem$x1^2, x22=chem$x2^2, 
  x12=chem$x1*chem$x2)
y <- chem$y
fit <- lm(y ~ ., data=X)
b <- coef(fit)[2:3]
B <- matrix(NA, nrow=2, ncol=2)
diag(B) <- coef(fit)[4:5]
B[1,2] <- B[2,1] <- coef(fit)[6]/2
E <- eigen(B)
lambda <- E$values
o <- order(abs(lambda), decreasing=TRUE)
P <- E$vectors[,o]
print(lambda <- lambda[o])


In [ ]:
U <- data.frame(cbind(chem$x1, chem$x2) %*% P)
names(U) <- c("u1", "u2")
U$u11 <- U$u1^2
U$u22 <- U$u2^2
fitU <- lm(y ~ ., data=U)


In [ ]:
rbind(dlr2=coef(fitU)[-1], eigen=c(b %*% P, lambda))



In [ ]:
summary(fitU)$coefficients[4:5,]



In [ ]:
confint(fitU)[4:5,]

